In [1]:
import pandas as pd
import numpy as np
import requests
import os
import shutil
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
mvps = pd.read_csv('data/mvp.csv')

In [3]:
mvps

,Unnamed: 0,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,0,1,David Robinson,29,SAS,73.0,901.0,1050,0.858,81,...,10.8,2.9,1.7,3.2,0.530,0.300,0.774,17.5,0.273,1995
1,1,2,Shaquille O'Neal,22,ORL,12.0,605.0,1050,0.576,79,...,11.4,2.7,0.9,2.4,0.583,0.000,0.533,14.0,0.230,1995
2,2,3,Karl Malone,31,UTA,14.0,532.0,1050,0.507,82,...,10.6,3.5,1.6,1.0,0.536,0.268,0.742,13.8,0.212,1995
3,3,4,Patrick Ewing,32,NYK,2.0,230.0,1050,0.219,79,...,11.0,2.7,0.9,2.0,0.503,0.286,0.750,9.6,0.157,1995
4,4,5,Hakeem Olajuwon,32,HOU,1.0,147.0,1050,0.140,72,...,10.8,3.5,1.8,3.4,0.517,0.188,0.756,10.7,0.181,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,10,11,Russell Westbrook,32,WAS,0.0,5.0,1010,0.005,65,...,11.5,11.7,1.4,0.4,0.439,0.315,0.656,3.7,0.075,2021
402,11,12,Ben Simmons,24,PHI,0.0,3.0,1010,0.003,58,...,7.2,6.9,1.6,0.6,0.557,0.300,0.613,6.0,0.153,2021
403,12,13T,James Harden,31,TOT,0.0,1.0,1010,0.001,44,...,7.9,10.8,1.2,0.8,0.466,0.362,0.861,7.0,0.208,2021
404,13,13T,LeBron James,36,LAL,0.0,1.0,1010,0.001,45,...,7.7,7.8,1.1,0.6,0.513,0.365,0.698,5.6,0.179,2021


In [4]:
mvps = mvps[['Player', 'Pts Won', 'Pts Max', 'Share', 'Year']]

In [5]:
mvps

,Player,Pts Won,Pts Max,Share,Year
0,David Robinson,901.0,1050,0.858,1995
1,Shaquille O'Neal,605.0,1050,0.576,1995
2,Karl Malone,532.0,1050,0.507,1995
3,Patrick Ewing,230.0,1050,0.219,1995
4,Hakeem Olajuwon,147.0,1050,0.140,1995
...,...,...,...,...,...
401,Russell Westbrook,5.0,1010,0.005,2021
402,Ben Simmons,3.0,1010,0.003,2021
403,James Harden,1.0,1010,0.001,2021
404,LeBron James,1.0,1010,0.001,2021


In [6]:
players = pd.read_csv("data/players.csv")

In [7]:
players

,Unnamed: 0,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,0,1,Alaa Abdelnaby,PF,26,TOT,54,0,9.4,2.2,...,0.7,1.4,2.1,0.2,0.3,0.2,0.8,1.9,4.7,1995
1,1,1,Alaa Abdelnaby,PF,26,SAC,51,0,9.3,2.3,...,0.7,1.4,2.1,0.3,0.3,0.2,0.8,2.0,5.0,1995
2,2,1,Alaa Abdelnaby,PF,26,PHI,3,0,10.0,0.3,...,1.0,1.7,2.7,0.0,0.0,0.0,1.7,0.7,0.7,1995
3,3,2,Mahmoud Abdul-Rauf,PG,25,DEN,73,43,28.5,6.5,...,0.4,1.4,1.9,3.6,1.1,0.1,1.6,1.7,16.0,1995
4,4,3,Michael Adams,PG,32,CHH,29,0,15.3,2.3,...,0.2,0.8,1.0,3.3,0.8,0.0,0.9,1.4,6.5,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16978,836,601,Thaddeus Young,PF,33,TOR,26,0,18.3,2.6,...,1.5,2.9,4.4,1.7,1.2,0.4,0.8,1.7,6.3,2022
16979,837,602,Trae Young,PG,23,ATL,76,76,34.9,9.4,...,0.7,3.1,3.7,9.7,0.9,0.1,4.0,1.7,28.4,2022
16980,838,603,Omer Yurtseven,C,23,MIA,56,12,12.6,2.3,...,1.5,3.7,5.3,0.9,0.3,0.4,0.7,1.5,5.3,2022
16981,839,604,Cody Zeller,C,29,POR,27,0,13.1,1.9,...,1.9,2.8,4.6,0.8,0.3,0.2,0.7,2.1,5.2,2022


In [8]:
players = players.drop(columns = ['Unnamed: 0', 'Rk'])

In [9]:
players['Player'] = players['Player'].str.replace('*','')

In [10]:
players

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,Alaa Abdelnaby,PF,26,TOT,54,0,9.4,2.2,4.3,.511,...,0.7,1.4,2.1,0.2,0.3,0.2,0.8,1.9,4.7,1995
1,Alaa Abdelnaby,PF,26,SAC,51,0,9.3,2.3,4.3,.532,...,0.7,1.4,2.1,0.3,0.3,0.2,0.8,2.0,5.0,1995
2,Alaa Abdelnaby,PF,26,PHI,3,0,10.0,0.3,3.7,.091,...,1.0,1.7,2.7,0.0,0.0,0.0,1.7,0.7,0.7,1995
3,Mahmoud Abdul-Rauf,PG,25,DEN,73,43,28.5,6.5,13.8,.470,...,0.4,1.4,1.9,3.6,1.1,0.1,1.6,1.7,16.0,1995
4,Michael Adams,PG,32,CHH,29,0,15.3,2.3,5.1,.453,...,0.2,0.8,1.0,3.3,0.8,0.0,0.9,1.4,6.5,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16978,Thaddeus Young,PF,33,TOR,26,0,18.3,2.6,5.5,.465,...,1.5,2.9,4.4,1.7,1.2,0.4,0.8,1.7,6.3,2022
16979,Trae Young,PG,23,ATL,76,76,34.9,9.4,20.3,.460,...,0.7,3.1,3.7,9.7,0.9,0.1,4.0,1.7,28.4,2022
16980,Omer Yurtseven,C,23,MIA,56,12,12.6,2.3,4.4,.526,...,1.5,3.7,5.3,0.9,0.3,0.4,0.7,1.5,5.3,2022
16981,Cody Zeller,C,29,POR,27,0,13.1,1.9,3.3,.567,...,1.9,2.8,4.6,0.8,0.3,0.2,0.7,2.1,5.2,2022


In [11]:
def single_player(dataframe):
    if dataframe.shape[0]==1:
        return dataframe
    else:
        row = dataframe[dataframe['Tm'] == 'TOT']
        row['Tm'] = dataframe.iloc[-1,:]['Tm']
        return row

In [12]:
players = players.groupby(['Player', 'Year']).apply(single_player)

In [13]:
players.head(50)

Player Pos Age   Tm   G  GS    MP   FG  \
Player         Year                                                         
A.C. Green     1995 165        A.C. Green  SF  31  PHO  82  52  32.8  3.8   
               1996 661        A.C. Green  SF  32  PHO  82  36  25.8  2.6   
               1997 1236       A.C. Green  PF  33  DAL  83  73  30.0  2.8   
               1998 1823       A.C. Green  PF  34  DAL  82  68  32.3  3.0   
               1999 2381       A.C. Green  PF  35  DAL  50  35  18.5  2.2   
               2000 2909       A.C. Green  PF  36  LAL  82  82  23.5  2.1   
               2001 3428       A.C. Green  PF  37  MIA  82   1  17.2  1.8   
A.J. Bramlett  2000 2782    A.J. Bramlett   C  23  CLE   8   0   7.6  0.5   
A.J. Guyton    2001 3431      A.J. Guyton  PG  22  CHI  33   8  19.1  2.4   
               2002 3984      A.J. Guyton  PG  23  CHI  45   6  13.5  2.0   
               2003 4491      A.J. Guyton  PG  24  GSW   2   0   4.5  0.0   
A.J. Hammons   2017 12912    A.J. Hammons   C  24  DAL  22   0   7.4  0.8   
A.J. Price     2010 8825       A.J. Price  PG  23  IND  56   2  15.4  2.6   
               2011 9454       A.J. Price  PG  24  IND  50   0  15.9  2.3   
               2012 10038      A.J. Price  PG  25  IND  44   1  12.9  1.3   
               2013 10630      A.J. Price  PG  26  WAS  57  22  22.4  2.8   
               2014 11260      A.J. Price  SG  27  MIN  28   0   3.5  0.7   
               2015 11908      A.J. Price  PG  28  PHO  26   0  12.5  2.0   
Aaron Brooks   2008 7229     Aaron Brooks  PG  23  HOU  51   0  11.9  1.8   
               2009 7850     Aaron Brooks  PG  24  HOU  80  35  25.0  4.0   
               2010 8451     Aaron Brooks  PG  25  HOU  82  82  35.6  7.0   
               2011 9067     Aaron Brooks  PG  26  PHO  59  12  21.8  3.7   
               2013 10266    Aaron Brooks  PG  28  HOU  53  20  18.8  2.7   
               2014 10874    Aaron Brooks  PG  29  DEN  72  12  21.6  3.2   
               2015 11498    Aaron Brooks  PG  30  CHI  82  21  23.0  4.2   
               2016 12163    Aaron Brooks  PG  31  CHI  69   0  16.1  2.7   
               2017 12766    Aaron Brooks  PG  32  IND  65   0  13.8  1.9   
               2018 13395    Aaron Brooks  PG  33  MIN  32   1   5.9  0.9   
Aaron Gordon   2015 11652    Aaron Gordon  PF  19  ORL  47   8  17.0  2.0   
               2016 12284    Aaron Gordon  PF  20  ORL  78  37  23.9  3.5   
               2017 12896    Aaron Gordon  SF  21  ORL  80  72  28.7  4.9   
               2018 13537    Aaron Gordon  PF  22  ORL  58  57  32.9  6.5   
               2019 14260    Aaron Gordon  PF  23  ORL  78  78  33.8  6.0   
               2020 14975    Aaron Gordon  PF  24  ORL  62  62  32.5  5.4   
               2021 15651    Aaron Gordon  PF  25  DEN  50  50  27.7  4.6   
               2022 16410    Aaron Gordon  PF  26  DEN  75  75  31.7  5.8   
Aaron Gray     2008 7369       Aaron Gray   C  23  CHI  61   1  10.0  1.7   
               2009 7990       Aaron Gray   C  24  CHI  56  18  12.8  1.4   
               2010 8580       Aaron Gray   C  25  NOH  32   0   9.7  1.3   
               2011 9231       Aaron Gray   C  26  NOH  41   6  13.0  1.4   
               2012 9828       Aaron Gray   C  27  TOR  49  40  16.6  1.7   
               2013 10393      Aaron Gray   C  28  TOR  42  16  12.2  1.1   
               2014 11029      Aaron Gray   C  29  SAC  37   6   9.6  0.7   
Aaron Harrison 2016 12320  Aaron Harrison  SG  21  CHO  21   0   4.4  0.2   
               2017 12924  Aaron Harrison  SG  22  CHO   5   0   3.4  0.0   
               2018 13567  Aaron Harrison  SG  23  DAL   9   3  25.9  2.1   
Aaron Henry    2022 16455     Aaron Henry  SF  22  PHI   6   0   2.8  0.2   
Aaron Holiday  2019 14313   Aaron Holiday  PG  22  IND  50   0  12.9  2.1   
               2020 15036   Aaron Holiday  PG  23  IND  66  33  24.5  3.5   
               2021 15715   Aaron Holiday  PG  24  IND  66   8  17.8  2.6   

                            FG

In [14]:
players.index = players.index.droplevel()

In [15]:
players.index = players.index.droplevel()

In [16]:
players

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
165,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,.504,...,2.4,5.8,8.2,1.5,0.7,0.4,1.4,1.8,11.2,1995
661,A.C. Green,SF,32,PHO,82,36,25.8,2.6,5.4,.484,...,2.0,4.7,6.8,0.9,0.5,0.3,1.0,1.7,7.5,1996
1236,A.C. Green,PF,33,DAL,83,73,30.0,2.8,5.8,.483,...,2.7,5.2,7.9,0.8,0.8,0.2,0.9,1.7,7.2,1997
1823,A.C. Green,PF,34,DAL,82,68,32.3,3.0,6.5,.453,...,2.7,5.5,8.1,1.5,1.0,0.3,0.8,1.9,7.3,1998
2381,A.C. Green,PF,35,DAL,50,35,18.5,2.2,5.1,.422,...,1.6,2.9,4.6,0.5,0.6,0.2,0.4,1.4,4.9,1999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4193,Željko Rebrača,C,29,DET,74,4,15.9,2.6,5.1,.505,...,1.1,2.8,3.9,0.5,0.4,1.0,1.1,2.6,6.9,2002
4693,Željko Rebrača,C,30,DET,30,12,16.3,2.7,4.8,.552,...,0.9,2.2,3.1,0.3,0.2,0.6,1.0,2.6,6.6,2003
5274,Željko Rebrača,C,31,ATL,24,2,11.4,1.4,3.2,.442,...,1.0,1.5,2.4,0.3,0.2,0.5,0.7,2.2,3.8,2004
5874,Željko Rebrača,C,32,LAC,58,2,16.0,2.3,4.0,.568,...,0.8,2.3,3.2,0.4,0.2,0.7,0.8,2.2,5.8,2005


In [17]:
df_combinado = players.merge(mvps, how = 'outer', on = ['Player', 'Year'])

In [18]:
df_combinado

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,AST,STL,BLK,TOV,PF,PTS,Year,Pts Won,Pts Max,Share
0,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,.504,...,1.5,0.7,0.4,1.4,1.8,11.2,1995,NaN,NaN,NaN
1,A.C. Green,SF,32,PHO,82,36,25.8,2.6,5.4,.484,...,0.9,0.5,0.3,1.0,1.7,7.5,1996,NaN,NaN,NaN
2,A.C. Green,PF,33,DAL,83,73,30.0,2.8,5.8,.483,...,0.8,0.8,0.2,0.9,1.7,7.2,1997,NaN,NaN,NaN
3,A.C. Green,PF,34,DAL,82,68,32.3,3.0,6.5,.453,...,1.5,1.0,0.3,0.8,1.9,7.3,1998,NaN,NaN,NaN
4,A.C. Green,PF,35,DAL,50,35,18.5,2.2,5.1,.422,...,0.5,0.6,0.2,0.4,1.4,4.9,1999,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13128,Željko Rebrača,C,29,DET,74,4,15.9,2.6,5.1,.505,...,0.5,0.4,1.0,1.1,2.6,6.9,2002,NaN,NaN,NaN
13129,Željko Rebrača,C,30,DET,30,12,16.3,2.7,4.8,.552,...,0.3,0.2,0.6,1.0,2.6,6.6,2003,NaN,NaN,NaN
13130,Željko Rebrača,C,31,ATL,24,2,11.4,1.4,3.2,.442,...,0.3,0.2,0.5,0.7,2.2,3.8,2004,NaN,NaN,NaN
13131,Željko Rebrača,C,32,LAC,58,2,16.0,2.3,4.0,.568,...,0.4,0.2,0.7,0.8,2.2,5.8,2005,NaN,NaN,NaN


In [19]:
df_combinado[['Pts Won', 'Pts Max', 'Share']] = df_combinado[['Pts Won', 'Pts Max', 'Share']].fillna(0)

In [20]:
df_combinado.head(50)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,AST,STL,BLK,TOV,PF,PTS,Year,Pts Won,Pts Max,Share
0,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,.504,...,1.5,0.7,0.4,1.4,1.8,11.2,1995,0.0,0.0,0.0
1,A.C. Green,SF,32,PHO,82,36,25.8,2.6,5.4,.484,...,0.9,0.5,0.3,1.0,1.7,7.5,1996,0.0,0.0,0.0
2,A.C. Green,PF,33,DAL,83,73,30.0,2.8,5.8,.483,...,0.8,0.8,0.2,0.9,1.7,7.2,1997,0.0,0.0,0.0
3,A.C. Green,PF,34,DAL,82,68,32.3,3.0,6.5,.453,...,1.5,1.0,0.3,0.8,1.9,7.3,1998,0.0,0.0,0.0
4,A.C. Green,PF,35,DAL,50,35,18.5,2.2,5.1,.422,...,0.5,0.6,0.2,0.4,1.4,4.9,1999,0.0,0.0,0.0
5,A.C. Green,PF,36,LAL,82,82,23.5,2.1,4.7,.447,...,1.0,0.6,0.2,0.6,1.5,5.0,2000,0.0,0.0,0.0
6,A.C. Green,PF,37,MIA,82,1,17.2,1.8,4.0,.444,...,0.5,0.4,0.1,0.5,1.5,4.5,2001,0.0,0.0,0.0
7,A.J. Bramlett,C,23,CLE,8,0,7.6,0.5,2.6,.190,...,0.0,0.1,0.0,0.4,1.6,1.0,2000,0.0,0.0,0.0
8,A.J. Guyton,PG,22,CHI,33,8,19.1,2.4,5.8,.406,...,1.9,0.3,0.2,0.7,1.1,6.0,2001,0.0,0.0,0.0
9,A.J. Guyton,PG,23,CHI,45,6,13.5,2.0,5.4,.361,...,1.8,0.2,0.2,0.8,0.5,5.4,2002,0.0,0.0,0.0


In [21]:
df_combinado = df_combinado.apply(pd.to_numeric, errors = 'ignore')

In [22]:
df_combinado.dtypes

Player      object
Pos         object
Age          int64
Tm          object
G            int64
GS           int64
MP         float64
FG         float64
FGA        float64
FG%        float64
3P         float64
3PA        float64
3P%        float64
2P         float64
2PA        float64
2P%        float64
eFG%       float64
FT         float64
FTA        float64
FT%        float64
ORB        float64
DRB        float64
TRB        float64
AST        float64
STL        float64
BLK        float64
TOV        float64
PF         float64
PTS        float64
Year         int64
Pts Won    float64
Pts Max    float64
Share      float64
dtype: object

In [23]:
df_combinado.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,AST,STL,BLK,TOV,PF,PTS,Year,Pts Won,Pts Max,Share
0,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,0.504,...,1.5,0.7,0.4,1.4,1.8,11.2,1995,0.0,0.0,0.0
1,A.C. Green,SF,32,PHO,82,36,25.8,2.6,5.4,0.484,...,0.9,0.5,0.3,1.0,1.7,7.5,1996,0.0,0.0,0.0
2,A.C. Green,PF,33,DAL,83,73,30.0,2.8,5.8,0.483,...,0.8,0.8,0.2,0.9,1.7,7.2,1997,0.0,0.0,0.0
3,A.C. Green,PF,34,DAL,82,68,32.3,3.0,6.5,0.453,...,1.5,1.0,0.3,0.8,1.9,7.3,1998,0.0,0.0,0.0
4,A.C. Green,PF,35,DAL,50,35,18.5,2.2,5.1,0.422,...,0.5,0.6,0.2,0.4,1.4,4.9,1999,0.0,0.0,0.0


In [24]:
nicknames = pd.read_csv('data/nicknames.txt')

In [25]:
nicknames

,Abbreviation,Name
0,ATL,Atlanta Hawks
1,BRK,Brooklyn Nets
2,BKN,Brooklyn Nets
3,BOS,Boston Celtics
4,CHA,Charlotte Bobcats
5,CHH,Charlotte Hornets
6,CHO,Charlotte Hornets
7,CHI,Chicago Bulls
8,CLE,Cleveland Cavaliers
9,DAL,Dallas Mavericks


In [33]:
nicknames = {}
with open("data/nicknames.txt") as f:
    lines = f.readlines()
    for line in lines[1:]:
        abreviado,name = line.replace("\n","").split(",")
        nicknames[abreviado] = name

In [34]:
df_combinado["Team"] = df_combinado["Tm"].map(nicknames)


In [35]:
df_combinado.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,STL,BLK,TOV,PF,PTS,Year,Pts Won,Pts Max,Share,Team
0,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,0.504,...,0.7,0.4,1.4,1.8,11.2,1995,0.0,0.0,0.0,Phoenix Suns
1,A.C. Green,SF,32,PHO,82,36,25.8,2.6,5.4,0.484,...,0.5,0.3,1.0,1.7,7.5,1996,0.0,0.0,0.0,Phoenix Suns
2,A.C. Green,PF,33,DAL,83,73,30.0,2.8,5.8,0.483,...,0.8,0.2,0.9,1.7,7.2,1997,0.0,0.0,0.0,Dallas Mavericks
3,A.C. Green,PF,34,DAL,82,68,32.3,3.0,6.5,0.453,...,1.0,0.3,0.8,1.9,7.3,1998,0.0,0.0,0.0,Dallas Mavericks
4,A.C. Green,PF,35,DAL,50,35,18.5,2.2,5.1,0.422,...,0.6,0.2,0.4,1.4,4.9,1999,0.0,0.0,0.0,Dallas Mavericks


In [36]:
teams = pd.read_csv('data/teams.csv')

In [37]:
teams

,Unnamed: 0,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,0,57,25,.695,—,110.9,103.8,6.44,1995,Orlando Magic*
1,1,55,27,.671,2.0,98.2,95.1,2.78,1995,New York Knicks*
2,2,35,47,.427,22.0,102.8,104.7,-1.92,1995,Boston Celtics*
3,3,32,50,.390,25.0,101.1,102.8,-1.85,1995,Miami Heat
4,4,30,52,.366,27.0,98.1,101.2,-3.28,1995,New Jersey Nets
...,...,...,...,...,...,...,...,...,...,...
908,13,42,30,.583,—,112.4,110.2,2.26,2021,Dallas Mavericks*
909,14,38,34,.528,4.0,113.3,112.3,1.07,2021,Memphis Grizzlies*
910,15,33,39,.458,9.0,111.1,112.8,-1.58,2021,San Antonio Spurs
911,16,31,41,.431,11.0,114.6,114.9,-0.20,2021,New Orleans Pelicans


In [38]:
teams = teams[~teams['W'].str.contains('Division')]

In [39]:
teams['Team'] = teams['Team'].str.replace('*', '')

/Users/jorgediazserrano/miniconda3/envs/ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
teams['GB'] = teams['GB'].replace('—', '0')

/Users/jorgediazserrano/miniconda3/envs/ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
teams = teams.drop(columns = ['Unnamed: 0'])

In [42]:
teams

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,57,25,.695,0,110.9,103.8,6.44,1995,Orlando Magic
1,55,27,.671,2.0,98.2,95.1,2.78,1995,New York Knicks
2,35,47,.427,22.0,102.8,104.7,-1.92,1995,Boston Celtics
3,32,50,.390,25.0,101.1,102.8,-1.85,1995,Miami Heat
4,30,52,.366,27.0,98.1,101.2,-3.28,1995,New Jersey Nets
...,...,...,...,...,...,...,...,...,...
908,42,30,.583,0,112.4,110.2,2.26,2021,Dallas Mavericks
909,38,34,.528,4.0,113.3,112.3,1.07,2021,Memphis Grizzlies
910,33,39,.458,9.0,111.1,112.8,-1.58,2021,San Antonio Spurs
911,31,41,.431,11.0,114.6,114.9,-0.20,2021,New Orleans Pelicans


In [43]:
df_final = df_combinado.merge(teams, how = 'outer', on = ['Team','Year'])

In [44]:
df_final

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,0.504,...,0.0,0.000,Phoenix Suns,59,23,.720,0,110.6,106.8,3.86
1,Aaron Swinson,SF,24,PHO,9,0,5.7,1.1,2.0,0.556,...,0.0,0.000,Phoenix Suns,59,23,.720,0,110.6,106.8,3.86
2,Antonio Lang,SF,22,PHO,12,0,4.4,0.3,0.8,0.400,...,0.0,0.000,Phoenix Suns,59,23,.720,0,110.6,106.8,3.86
3,Charles Barkley,PF,31,PHO,68,66,35.0,8.1,16.8,0.486,...,1050.0,0.091,Phoenix Suns,59,23,.720,0,110.6,106.8,3.86
4,Dan Majerle,SF,29,PHO,82,46,37.7,5.3,12.6,0.425,...,0.0,0.000,Phoenix Suns,59,23,.720,0,110.6,106.8,3.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13128,Spencer Hawes,PF,28,MIL,54,1,14.8,2.5,5.1,0.484,...,0.0,0.000,Milwaukee Bucks,42,40,.512,9.0,103.6,103.8,-0.45
13129,Steve Novak,PF,33,MIL,8,0,2.8,0.3,0.9,0.286,...,0.0,0.000,Milwaukee Bucks,42,40,.512,9.0,103.6,103.8,-0.45
13130,Terrence Jones,PF,25,MIL,54,12,23.5,4.3,9.1,0.470,...,0.0,0.000,Milwaukee Bucks,42,40,.512,9.0,103.6,103.8,-0.45
13131,Thon Maker,C,19,MIL,57,34,9.9,1.5,3.2,0.459,...,0.0,0.000,Milwaukee Bucks,42,40,.512,9.0,103.6,103.8,-0.45


In [45]:
df_final = df_final.apply(pd.to_numeric, errors = 'ignore')

In [46]:
df_final.dtypes

Player      object
Pos         object
Age          int64
Tm          object
G            int64
GS           int64
MP         float64
FG         float64
FGA        float64
FG%        float64
3P         float64
3PA        float64
3P%        float64
2P         float64
2PA        float64
2P%        float64
eFG%       float64
FT         float64
FTA        float64
FT%        float64
ORB        float64
DRB        float64
TRB        float64
AST        float64
STL        float64
BLK        float64
TOV        float64
PF         float64
PTS        float64
Year         int64
Pts Won    float64
Pts Max    float64
Share      float64
Team        object
W          float64
L          float64
W/L%       float64
GB         float64
PS/G       float64
PA/G       float64
SRS        float64
dtype: object

In [47]:
df_final.to_csv('data/player_stats.csv')